In [9]:
import pandas as pd
import numpy as np

In [10]:
# Load the datasets using the specified paths
job_posting = pd.read_csv('./Raw_Data/postings.csv')
companies = pd.read_csv('./Raw_Data/companies/companies.csv')
job_skills = pd.read_csv('./Raw_Data/jobs/job_skills.csv')
mapping_skills = pd.read_csv('./Raw_Data/mappings/skills.csv')

In [40]:
# Ensure 'company_id' is a nullable integer and 'company_size' is a float in the companies DataFrame
companies['company_id'] = companies['company_id'].astype('Int64')  # Sets as nullable integer for missing values
companies['company_size'] = pd.to_numeric(companies['company_size'], errors='coerce')  # Converts to float, setting non-numeric values as NaN


In [42]:
# Merge job_skills with mapping_skills to include full skill names
job_skills_detailed = job_skills.merge(mapping_skills, on="skill_abr", how="left")
# Display the first few rows of the merged data for verification
job_skills_detailed.head()


,job_id,skill_abr,skill_name
0,3884428798,MRKT,Marketing
1,3884428798,PR,Public Relations
2,3884428798,WRT,Writing/Editing
3,3887473071,SALE,Sales
4,3887465684,FIN,Finance


In [41]:
# Check for missing values in each DataFrame
missing_values_companies = companies.isnull().sum()
missing_values_job_skills = job_skills.isnull().sum()
missing_values_skills = mapping_skills.isnull().sum()
missing_values_postings = job_posting.isnull().sum()

In [43]:
# Display summaries of missing values after handling for verification
print("Missing values in companies:\n", missing_values_companies)
print("\nMissing values in job_skills:\n", missing_values_job_skills)
print("\nMissing values in mapping_skills:\n", missing_values_skills)
print("\nMissing values in job_posting:\n", missing_values_postings)


Missing values in companies:
 company_id      0
name            0
description     0
company_size    0
state           0
country         0
city            0
zip_code        0
address         0
url             0
dtype: int64

Missing values in job_skills:
 job_id       0
skill_abr    0
dtype: int64

Missing values in mapping_skills:
 skill_abr     0
skill_name    0
dtype: int64

Missing values in job_posting:
 job_id                             0
company_name                    1719
title                              0
description                        7
max_salary                     94056
pay_period                     87776
location                           0
company_id                      1717
views                           1689
med_salary                    117569
min_salary                     94056
formatted_work_type                0
applies                       100529
original_listed_time               0
remote_allowed                108603
job_posting_url                  

In [45]:
# Fill missing values in the companies DataFrame
companies['company_size'].fillna(companies['company_size'].median(), inplace=True)
companies['description'].fillna("No description provided", inplace=True)
companies['state'].fillna("Unknown", inplace=True)
companies['city'].fillna("Unknown", inplace=True)
companies['zip_code'].fillna("Unknown", inplace=True)
companies['address'].fillna("Unknown", inplace=True)

# Drop rows in postings where 'company_id' is missing and create a separate copy
postings = job_posting.dropna(subset=['company_id']).copy()

# Convert 'company_id' to nullable integer to avoid float conversion
postings['company_id'] = postings['company_id'].astype('Int64')


In [47]:
# Print initial missing values in postings
print("Initial missing values in postings:\n", postings.isnull().sum())



Initial missing values in postings:
 job_id                             0
company_name                       2
title                              0
description                        6
max_salary                     92793
pay_period                     86568
location                           0
company_id                         0
views                           1627
med_salary                    115907
min_salary                     92793
formatted_work_type                0
applies                        99236
original_listed_time               0
remote_allowed                107318
job_posting_url                    0
application_url                35063
application_type                   0
expiry                             0
closed_time                   121076
formatted_experience_level     27949
skills_desc                   119696
listed_time                        0
posting_domain                 38252
sponsored                          0
work_type                          0
c

In [ ]:
# Fill key missing values in postings
postings['company_name'].fillna("Unknown Company", inplace=True)
postings['description'].fillna("No description provided", inplace=True)

# Optionally, fill salary-related columns with median salary values
salary_median = postings[['max_salary', 'min_salary']].median()
postings['max_salary'].fillna(salary_median['max_salary'], inplace=True)
postings['min_salary'].fillna(salary_median['min_salary'], inplace=True)

# Optionally drop columns with too many missing values (e.g., if over 80% missing)
# Drop columns if they exist; ignore any that aren't in the DataFrame
postings.drop(columns=['med_salary', 'skills_desc', 'remote_allowed', 'closed_time'], inplace=True, errors='ignore')



In [64]:
# Integration Step
job_data_combined = postings.merge(job_skills_detailed, on="job_id", how="left")
job_data_combined['company_id'] = job_data_combined['company_id'].astype('Int64')  # Ensure alignment with companies

# Now merge job_data_combined with companies on 'company_id' to include company information
full_data = job_data_combined.merge(companies, on="company_id", how="left")

# Check final missing values in full_data
print("Final missing values in full_data:\n", full_data.isnull().sum())


Final missing values in full_data:
 job_id                             0
company_name                       0
title                              0
description_x                      0
max_salary                         0
pay_period                    144438
location                           0
company_id                         0
views                           2802
min_salary                         0
formatted_work_type                0
applies                       165087
original_listed_time               0
job_posting_url                    0
application_url                60051
application_type                   0
expiry                             0
formatted_experience_level     45677
listed_time                        0
posting_domain                 66789
sponsored                          0
work_type                          0
currency                      144438
compensation_type             144438
normalized_salary             144438
zip_code_x                     35322
fi

In [65]:
# Aggregation examples
job_count_per_company = full_data.groupby("company_id")['job_id'].count().reset_index()
job_count_per_company.rename(columns={"job_id": "total_jobs"}, inplace=True)
print("Total job postings per company:\n", job_count_per_company.head())

Total job postings per company:
    company_id  total_jobs
0        1009          33
1        1016          85
2        1025          24
3        1028         127
4        1033          37


In [66]:
skill_count = job_skills_detailed['skill_name'].value_counts().reset_index()
skill_count.columns = ['skill_name', 'count']
print("Top required skills across all jobs:\n", skill_count.head())

Top required skills across all jobs:
                skill_name  count
0  Information Technology  26137
1                   Sales  22475
2              Management  20861
3           Manufacturing  18185
4    Health Care Provider  17369


In [67]:
# Drop duplicates
full_data.drop_duplicates(inplace=True)

In [68]:
# Final sampling step
sampled_data = full_data.sample(frac=0.05, random_state=42).copy()
print("\nSample of fully processed data:\n", sampled_data.head())


Sample of fully processed data:
             job_id                                   company_name  \
179699  3905351309                           Signature HealthCARE   
20013   3887593646                           Keck Medicine of USC   
23240   3887872935                                        Netflix   
151463  3904577325  TMAC's Direct Hire (Executive Search) Service   
131876  3903840980                             FirstChoice Hiring   

                                                    title  \
179699                  Certified Nursing Assistant (CNA)   
20013   Licensed Vocational Nurse - Emergency Room - F...   
23240   Technical Program Manager 6 - Game SDK and Ser...   
151463     Field Medical Director, Pediatric Gene Editing   
131876       HVAC Technicians / Installers - Capital Heat   

                                            description_x  max_salary  \
179699  About Us\n\nWelcome to Signature Healthcare of...     80000.0   
20013   The Licensed Vocational Nurse 